# Príklad agenta na rezerváciu hotelov a letov

Toto riešenie vám pomôže rezervovať letenky a hotel. Scenár je cesta z letiska London Heathrow LHR 20. februára 2024 do New York JFK s návratom 27. februára 2025, let v ekonomickej triede iba s British Airways. Chcem pobyt v hoteli Hilton v New Yorku, prosím, poskytnite ceny za let a hotel.


# Inicializujte službu Azure AI Agent a získajte konfiguračné informácie z **.env**

### **.env**

Vytvorte súbor .env

**.env** obsahuje reťazec pripojenia služby Azure AI Agent, model používaný AOAI a príslušnú službu Google API Search, ENDPOINT a pod.

- **AZURE_AI_AGENT_MODEL_DEPLOYMENT_NAME** = "Názov nasadenia modelu služby Azure AI Agent"

[**POZNÁMKA**] Budete potrebovať model s 100 000 limitom rýchlosti (tokeny za minútu) a limitom 600 (požiadaviek za minútu)

  Model môžete získať v Microsoft Foundry - Model a Endpoint.

- **AZURE_AI_AGENT_PROJECT_CONNECTION_STRING** = "Reťazec pripojenia projektu služby Azure AI Agent"

  Reťazec pripojenia projektu nájdete v prehľade svojho projektu na obrazovke AI Foundry Portal.

- **SERPAPI_SEARCH_API_KEY** = "Váš SERPAPI Search API KĽÚČ"
- **SERPAPI_SEARCH_ENDPOINT** = "Váš SERPAPI Search Endpoint"

Na získanie názvu nasadenia modelu a reťazca pripojenia projektu služby Azure AI Agent musíte vytvoriť službu Azure AI Agent. Odporúča sa použiť [túto šablónu](https://portal.azure.com/#create/Microsoft.Template/uri/https%3A%2F%2Fraw.githubusercontent.com%2Ffosteramanda%2Fazure-agent-quickstart-templates%2Frefs%2Fheads%2Fmaster%2Fquickstarts%2Fmicrosoft.azure-ai-agent-service%2Fstandard-agent%2Fazuredeploy.json) na jej priame vytvorenie (***Poznámka:*** Služba Azure AI Agent je momentálne nastavená iba v obmedzenej oblasti. Odporúča sa, aby ste sa riadili [týmto odkazom](https://learn.microsoft.com/en-us/azure/ai-services/agents/concepts/model-region-support) na nastavenie regiónu)

Agent potrebuje prístup k SERPAPI. Odporúča sa zaregistrovať sa pomocou [tohto odkazu](https://serpapi.com/searches). Po registrácii môžete získať jedinečný API KĽÚČ a ENDPOINT.


# Nastavenie 

Ak chcete spustiť tento notebook, musíte sa uistiť, že máte nainštalované potrebné knižnice pomocou príkazu `pip install -r requirements.txt`.


In [ ]:
from semantic_kernel import __version__

__version__

Vaša verzia Semantic Kernel by mala byť aspoň 1.27.2.


Načítajte svoje nastavenia a zdroje zo súboru .env, prosím, uistite sa, že ste pridali svoje kľúče a nastavenia a vytvorili lokálny súbor .env.


In [ ]:
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Prihlásiť sa do Azure

Teraz sa musíte prihlásiť do Azure. Otvorte terminál a spustite nasledujúci príkaz:

```bash
az login
```

Tento príkaz vás vyzve na zadanie vašich prihlasovacích údajov do Azure, čo umožní správnu funkčnosť služby Azure AI Agent.


# Vysvetlenie:
Toto je premenná, ktorá ukladá API kľúč pre prístup k SERP (Search Engine Results Page) API službe. API kľúč je jedinečný identifikátor používaný na overenie požiadaviek spojených s vaším účtom.

Účel: Účelom tohto riadku je uložiť API kľúč do premennej, aby mohol byť použitý na overenie požiadaviek do SERP API služby. API kľúč je potrebný na prístup k službe a vykonávanie vyhľadávaní.
Ako získať SERP API kľúč: Ak chcete získať SERP API kľúč, postupujte podľa týchto všeobecných krokov na stránke https://serpapi.com (presné kroky sa môžu líšiť v závislosti od konkrétnej SERP API služby, ktorú používate):

Vyberte SERP API službu: K dispozícii je niekoľko SERP API služieb, ako napríklad SerpAPI, Google Custom Search JSON API a ďalšie. Vyberte tú, ktorá najlepšie vyhovuje vašim potrebám.

Zaregistrujte si účet: Prejdite na webovú stránku vybranej SERP API služby a zaregistrujte si účet. Možno budete musieť poskytnúť niektoré základné informácie a overiť svoju emailovú adresu.

Vytvorte API kľúč: Po registrácii sa prihláste do svojho účtu a prejdite do sekcie API alebo do ovládacieho panela. Hľadajte možnosť vytvoriť alebo generovať nový API kľúč.
Skopírujte API kľúč do vášho súboru .env.


In [ ]:
SERP_API_KEY='SERPAPI_SEARCH_API_KEY'

# Explanation:
BASE_URL: Toto je premenná, ktorá uchováva základnú URL pre koncový bod SERP API. Názov premennej BASE_URL je konvencia používaná na označenie, že táto URL je východiskovým bodom pre vytváranie API požiadaviek.
'https://serpapi.com/search':

Toto je skutočný reťazec URL priradený premennej BASE_URL. Predstavuje koncový bod na vykonávanie vyhľadávacích dopytov pomocou SERP API.

# Purpose:
Účelom tohto riadku je definovať konštantu, ktorá obsahuje základnú URL pre SERP API. Táto URL bude použitá ako východiskový bod na zostavenie API požiadaviek na vykonávanie vyhľadávacích operácií.

# Usage:
Definovaním základnej URL v premennej ju môžete jednoducho znovu použiť vo vašom kóde vždy, keď potrebujete vykonať požiadavky na SERP API. To robí váš kód ľahšie udržiavateľným a znižuje riziko chýb spôsobených tvrdým zakódovaním URL na viacerých miestach. Aktuálny príklad je https://serpapi.com/search?engine=bing, ktorý používa Bing vyhľadávacie API. Rôzne API je možné vybrať na stránke https://Serpapi.com


In [ ]:
BASE_URL = 'https://serpapi.com/search?engine=bing'

# Vysvetlenie:

Tu sa nachádza váš kód pluginu.

Definícia triedy: `class BookingPlugin`: Definuje triedu pomenovanú BookingPlugin, ktorá obsahuje metódy na rezerváciu hotelov a letov.

Metóda rezervácie hotela:

- `@kernel_function(description="booking hotel")`: Dekorátor, ktorý popisuje funkciu ako jadrovú funkciu na rezerváciu hotelov.
- `def booking_hotel(self, query: Annotated[str, "The name of the city"], check_in_date: Annotated[str, "Hotel Check-in Time"], check_out_date: Annotated[str, "Hotel Check-out Time"]) -> Annotated[str, "Return the result of booking hotel information"]:`: Definuje metódu na rezerváciu hotelov s anotovanými parametrami a návratovým typom.

Metóda vytvára slovník parametrov pre požiadavku rezervácie hotela a posiela GET požiadavku na SERP API. Kontroluje stav odpovede a ak je úspešná, vráti vlastnosti hotela, inak None.

Metóda rezervácie letu: 

- `@kernel_function(description="booking flight")`: Dekorátor, ktorý popisuje funkciu ako jadrovú funkciu na rezerváciu letov.
- `def booking_flight(self, origin: Annotated[str, "The name of Departure"], destination: Annotated[str, "The name of Destination"], outbound_date: Annotated[str, "The date of outbound"], return_date: Annotated[str, "The date of Return_date"]) -> Annotated[str, "Return the result of booking flight information"]:`: Definuje metódu na rezerváciu letov s anotovanými parametrami a návratovým typom.

Metóda vytvára slovníky parametrov pre požiadavky na odletový a spiatočný let a posiela GET požiadavky na SERP API. Kontroluje stav odpovede a ak je úspešná, pridá informácie o lete do výsledného reťazca, inak vypíše chybové hlásenie. Metóda vráti výsledný reťazec obsahujúci informácie o lete.


In [ ]:
import requests

from typing import Annotated

from semantic_kernel.functions import kernel_function

# Define Booking Plugin
class BookingPlugin:
    """Booking Plugin for customers"""

    @kernel_function(description="booking hotel")
    def booking_hotel(
        self, 
        query: Annotated[str, "The name of the city"], 
        check_in_date: Annotated[str, "Hotel Check-in Time"], 
        check_out_date: Annotated[str, "Hotel Check-out Time"],
    ) -> Annotated[str, "Return the result of booking hotel information"]:
        """
        Function to book a hotel.
        Parameters:
        - query: The name of the city
        - check_in_date: Hotel Check-in Time
        - check_out_date: Hotel Check-out Time
        Returns:
        - The result of booking hotel information
        """

        # Define the parameters for the hotel booking request
        params = {
            "engine": "google_hotels",
            "q": query,
            "check_in_date": check_in_date,
            "check_out_date": check_out_date,
            "adults": "1",
            "currency": "GBP",
            "gl": "uk",
            "hl": "en",
            "api_key": SERP_API_KEY
        }

        # Send the GET request to the SERP API
        response = requests.get(BASE_URL, params=params)

        # Check if the request was successful
        if response.status_code == 200:
            # Parse the response content as JSON
            response = response.json()
            # Return the properties from the response
            return response["properties"]
        else:
            # Return None if the request failed
            return None

    @kernel_function(description="booking flight")
    def booking_flight(
        self, 
        origin: Annotated[str, "The name of Departure"], 
        destination: Annotated[str, "The name of Destination"], 
        outbound_date: Annotated[str, "The date of outbound"], 
        return_date: Annotated[str, "The date of Return_date"],
    ) -> Annotated[str, "Return the result of booking flight information"]:
        """
        Function to book a flight.
        Parameters:
        - origin: The name of Departure
        - destination: The name of Destination
        - outbound_date: The date of outbound
        - return_date: The date of Return_date
        - airline: The preferred airline carrier
        - hotel_brand: The preferred hotel brand
        Returns:
        - The result of booking flight information
        """
        
        # Define the parameters for the outbound flight request
        go_params = {
            "engine": "google_flights",
            "departure_id": "destination",
            "arrival_id": "origin",
            "outbound_date": "outbound_date",
            "return_date": "return_date",
            "currency": "GBP",
            "hl": "en",
            "airline": "airline",
            "hotel_brand": "hotel_brand",
            "api_key": "SERP_API_KEY"
        }
 
        print(go_params)

        # Send the GET request for the outbound flight
        go_response = requests.get(BASE_URL, params=go_params)

        # Initialize the result string
        result = ''

        # Check if the outbound flight request was successful
        if go_response.status_code == 200:
            # Parse the response content as JSON
            response = go_response.json()
            # Append the outbound flight information to the result
            result += "# outbound \n " + str(response)
        else:
            # Print an error message if the request failed
            print('error!!!')

        # Define the parameters for the return flight request
        back_params = {
            #"engine": "google_flights",
            "departure_id": destination,
            "arrival_id": origin,
            "outbound_date": outbound_date,
            "return_date": return_date,
            "currency": "GBP",
            "hl": "en",
            "api_key": SERP_API_KEY
        }

        # Send the GET request for the return flight
        back_response = requests.get(BASE_URL, params=back_params)

        # Check if the return flight request was successful
        if back_response.status_code == 200:
            # Parse the response content as JSON
            response = back_response.json()
            # Append the return flight information to the result
            result += "\n # return \n" + str(response)
        else:
            # Print an error message if the request failed
            print('error!!!')

        # Print the result
        print(result)

        # Return the result
        return result


# Vysvetlenie:
Importné príkazy: Importujú sa potrebné moduly pre Azure poverenia, AI agenta, obsah chatovej správy, rolu autora a dekorátor funkcie kernelu.

Asynchrónny správca kontextu: async with (DefaultAzureCredential() as creds, AzureAIAgent.create_client(credential=creds, conn_str="...") as client,): Nastavuje asynchrónneho správcu kontextu na spracovanie Azure poverení a vytvorenie klienta AI agenta.

Meno agenta a inštrukcie:
- `AGENT_NAME = "BookingAgent"`: Definuje meno agenta.
- `AGENT_INSTRUCTIONS = """..."""`: Poskytuje podrobné inštrukcie pre agenta, ako spracovať požiadavky na rezerváciu.

Vytvorenie definície agenta: `agent_definition = await client.agents.create_agent(...)`: Vytvorí definíciu agenta so zadaným modelom, menom a inštrukciami.

Vytvorenie AzureAI agenta: `agent = AzureAIAgent(...)`: Vytvorí AzureAI agenta pomocou klienta, definície agenta a zadaného pluginu.

Vytvorenie vláknu: `thread: AzureAIAgentThread | None = None`: Vytvorí vlákno pre agenta. Nie je potrebné najskôr vytvoriť vlákno - ak sa poskytne hodnota `None`, počas prvej invokácie sa vytvorí nové vlákno a vráti ako súčasť odpovede.

Vstupy od používateľa: `user_inputs = ["..."]`: Definuje zoznam vstupov od používateľa, ktoré má agent spracovať.

V bloku finally sa odstránia vlákno a agent, aby sa uvoľnili zdroje.


# Overovanie identity

Trieda `DefaultAzureCredential` je súčasťou Azure SDK pre Python. Poskytuje predvolený spôsob overenia identity v službách Azure. Pokúša sa overiť pomocou viacerých metód v konkrétnom poradí, ako sú premenné prostredia, spravovaná identita a poverenia Azure CLI.

Asynchrónne operácie: Modul aio naznačuje, že trieda DefaultAzureCredential podporuje asynchrónne operácie. To znamená, že ju môžete použiť s asyncio na vykonávanie neblokujúcich požiadaviek na overenie identity.


In [ ]:
# Import necessary modules
from azure.identity.aio import DefaultAzureCredential
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentSettings, AzureAIAgentThread

ai_agent_settings = AzureAIAgentSettings.create()

# Azure AI Setting
async with (
     DefaultAzureCredential() as creds,
    AzureAIAgent.create_client(
        credential=creds,
        conn_str=ai_agent_settings.project_connection_string.get_secret_value(),
    ) as client,
):    
    
    # Define the agent's name and instructions
    AGENT_NAME = "BookingAgent"
    AGENT_INSTRUCTIONS = """
    You are a booking agent, help me to book flights or hotels.

    Thought: Understand the user's intention and confirm whether to use the reservation system to complete the task.

    Action:
    - If booking a flight, convert the departure name and destination name into airport codes.
    - If booking a hotel or flight, use the corresponding API to call. Ensure that the necessary parameters are available. If any parameters are missing, use default values or assumptions to proceed.
    - If it is not a hotel or flight booking, respond with the final answer only.
    - Output the results using a markdown table:
    - For flight bookings, separate the outbound and return contents and list them in the order of Departure_airport Name | Airline | Flight Number | Departure Time | Arrival_airport Name | Arrival Time | Duration | Airplane | Travel Class | Price (USD) | Legroom | Extensions | Carbon Emissions (kg).
    - For hotel bookings, list them in the order of Properties Name | Properties description | check_in_time | check_out_time | prices | nearby_places | hotel_class | gps_coordinates.
    """

    # Create agent definition with the specified model, name, and instructions
    agent_definition = await client.agents.create_agent(
        model=ai_agent_settings.model_deployment_name,
        name=AGENT_NAME,
        instructions=AGENT_INSTRUCTIONS,
    )

    # Create the AzureAI Agent using the client and agent definition
    agent = AzureAIAgent(
        client=client,
        definition=agent_definition,
        plugins=[BookingPlugin()]
    )

    # Create a new thread for the agent
    # If no thread is provided, a new thread will be
    # created and returned with the initial response
    thread: AzureAIAgentThread | None = None

    # This is your prompt for the activity or task you want to complete 
    # Define user inputs for the agent to process we have provided some example prompts to test and validate 
    user_inputs = [
        # "Can you tell me the round-trip air ticket from  London to New York JFK aiport, the departure time is February 17, 2025, and the return time is February 23, 2025"
        # "Book a hotel in New York from Feb 20,2025 to Feb 24,2025"
        "Help me book flight tickets and hotel for the following trip London Heathrow LHR Feb 20th 2025 to New York JFK returning Feb 27th 2025 flying economy with British Airways only. I want a stay in a Hilton hotel in New York please provide costs for the flight and hotel"
        # "I have a business trip from London LHR to New York JFK on Feb 20th 2025 to Feb 27th 2025, can you help me to book a hotel and flight tickets"
    ]

    try:
        # Process each user input
        for user_input in user_inputs:
            print(f"# User: '{user_input}'")
            # Get the agent's response for the specified thread
            response = await agent.get_response(
                messages=user_input,
                thread=thread,
            )
            thread = response.thread
            # Print the agent's response
            print(f"{response.name}: '{response.content}'")
    finally:
        # Clean up by deleting the thread and agent
        await thread.delete() if thread else None
        await client.agents.delete_agent(agent.id)

---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Vyhlásenie o zodpovednosti**:  
Tento dokument bol preložený pomocou AI prekladateľskej služby [Co-op Translator](https://github.com/Azure/co-op-translator). Aj keď sa usilujeme o presnosť, vezmite prosím na vedomie, že automatické preklady môžu obsahovať chyby alebo nepresnosti. Pôvodný dokument v jeho rodnom jazyku by mal byť považovaný za rozhodujúci zdroj. Pre kritické informácie sa odporúča profesionálny ľudský preklad. Nie sme zodpovední za akékoľvek nedorozumenia alebo nesprávne interpretácie vyplývajúce z použitia tohto prekladu.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
